# Prepare

In [1]:
import json
import re
import os, sys
import pickle

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm

In [3]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [4]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
def clean_sentence(sent):
    sent = sent.strip()
    sent = re.sub('{laughing}|{clearing}|{singing}|{applauding}', '', sent)
    sent = re.sub('[(][(].*?[)][)]|-.*?-', '', sent)
    return sent

In [6]:
get_main_topic = lambda x: x.split(' > ')[0]

def load_data():
    file_list = [f_name for f_name in os.listdir('data') if f_name[-5:] == '.json']
    
    total_data = []
    for f_name in tqdm(file_list):
        with open('data/%s' %f_name, 'r') as f:
            data = json.loads(f.read())['document'][0]
            metadata = data['metadata']
            utterance = data['utterance']
            
            topic = get_main_topic(metadata['topic'])
            if topic[:4] == 'NWRW':
                continue

            last_speaker = None
            seg1 = seg2 = ''
            for u in utterance:
                if last_speaker is None:
                    last_speaker = u['speaker_id']
                    seg2 = u['form']
                elif last_speaker == u['speaker_id']:
                    seg2 += ' ' + u['original_form']
                else:
                    if seg1 and seg2:
                        total_data.append([f_name, topic, clean_sentence(seg1), clean_sentence(seg2)])
                    last_speaker = u['speaker_id']
                    seg1 = seg2
                    seg2 = u['original_form']
            if seg1 and seg2:
                total_data.append([f_name, topic, clean_sentence(seg1), clean_sentence(seg2)])
                
    return np.array(total_data)

In [7]:
device = torch.device("cuda:0")

In [8]:
bertmodel, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model
using cached model
using cached model


In [22]:
## Setting parameters
pre_max_len = 4096
max_len = 512
batch_size = 6
warmup_ratio = 0.1
num_epochs = 25
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [10]:
indices = np.indices([pre_max_len])[0]
def pad_transform(transform, sent1, sent2, max_len):
    tokens, valid_len, segs = transform([sent1, sent2])
    if valid_len < max_len:
        return tokens[:max_len], np.array((tokens != 1).sum(), dtype='int32'), segs[:max_len]
    
    idx2 = indices[segs == 1][:-1]
    idx1 = indices[:idx2[0]][1:-1]
    if len(idx1) < len(idx2) and (len(idx1) * 2 + 3) < max_len:
        idx2 = idx2[:(max_len - 3 - len(idx1))]
    elif len(idx2) < len(idx1) and (len(idx2) * 2 + 3) < max_len:
        idx1 = idx1[-(max_len - 3 - len(idx2)):]
    else:
        if len(idx1) < len(idx2):
            idx1, idx2 = idx1[-((max_len - 3) // 2):], idx2[:((max_len - 3) // 2 + 1)]
        else:
            idx1, idx2 = idx1[-((max_len - 3) // 2 + 1):], idx2[:((max_len - 3) // 2)]
    return (np.concatenate([[2], tokens[idx1], [3], tokens[idx2],[3]]), 
             np.array(max_len, dtype='int32'), 
             np.array([0] * (len(idx1) + 2) + [1] * (len(idx2) + 1)))

In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx1, sent_idx2, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=pre_max_len, pad=pad, pair=pair)

        self.sentences = [pad_transform(transform, d[sent_idx1], d[sent_idx2], max_len) for d in tqdm(dataset)]
#         self.sentences = [transform([d[sent_idx1], d[sent_idx2]]) for d in tqdm(dataset)]
        self.labels = [np.int32(d[label_idx]) for d in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [12]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        ret = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        pooler = ret[1]
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

# Train

In [13]:
def row_to_list(row):
    return str(row[2]), str(row[3]), label_w2i[row[1]]

In [36]:
raw_dataset = load_data()
file_with_label = np.unique(raw_dataset[:, :2], axis=0)
_labels, _counts = np.unique(file_with_label[:, 1], return_counts=True)
label_counts = dict(zip(_labels, _counts))

100%|██████████| 2232/2232 [00:02<00:00, 1062.10it/s]


In [37]:
np.random.seed(0)
np.random.shuffle(file_with_label)

valid_list = {label: [] for label in label_counts}
test_list = {label: [] for label in label_counts}

for file in file_with_label:
    if len(valid_list[file[1]]) < label_counts[file[1]] * 0.1:
        valid_list[file[1]].append(file[0])
    elif len(test_list[file[1]]) < label_counts[file[1]] * 0.1:
        test_list[file[1]].append(file[0])
        
_valid_list = []
_test_list = []
for label in label_counts:
    _valid_list.extend(valid_list[label])
    _test_list.extend(test_list[label])
    
valid_list = _valid_list
test_list = _test_list

In [38]:
raw_train_dataset, raw_valid_dataset, raw_test_dataset = [], [], []
for row in raw_dataset:
    if row[0] in valid_list:
        raw_valid_dataset.append(row)
    elif row[0] in test_list:
        raw_test_dataset.append(row)
    else:
        raw_train_dataset.append(row)
raw_train_dataset, raw_valid_dataset, raw_test_dataset = np.array(raw_train_dataset), np.array(raw_valid_dataset), np.array(raw_test_dataset)

In [48]:
np.savez('raw_train_valid_test.npz', raw_train_dataset, raw_valid_dataset, raw_test_dataset)

In [14]:
npz_file = np.load('raw_train_valid_test.npz')
raw_train_dataset, raw_valid_dataset, raw_test_dataset = npz_file['arr_0'], npz_file['arr_1'], npz_file['arr_2']

In [15]:
label_i2w = np.unique(raw_train_dataset[:, 1]).tolist()
label_w2i = {w: i for i, w in enumerate(label_i2w)}

In [22]:
label_w2i = pickle.load(open('ver_3.w2i', 'rb'))
label_i2w = {label_w2i[l]: l for l in label_w2i}

In [16]:
train_dataset = [row_to_list(r) for r in raw_train_dataset]
valid_dataset = [row_to_list(r) for r in raw_valid_dataset]
test_dataset = [row_to_list(r) for r in raw_test_dataset]

In [17]:
data_train = BERTDataset(train_dataset, 0, 1, 2, tok, max_len, True, True)
data_valid = BERTDataset(valid_dataset, 0, 1, 2, tok, max_len, True, True)
data_test = BERTDataset(test_dataset, 0, 1, 2, tok, max_len, True, True)

100%|██████████| 2277/2277 [00:04<00:00, 483.96it/s]


In [18]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
valid_dataloader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [19]:
model = BERTClassifier(bertmodel, dr_rate=0.5, num_classes=len(label_i2w)).to(device)

In [20]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
# loss_fn = nn.CrossEntropyLoss(weight=weight)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [21]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [24]:
pickle.dump(label_w2i, open('ver_3.w2i', 'wb'))
result = []

n_batch = len(train_dataloader)
for e in range(num_epochs):
    train_acc = 0.0
    valid_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader, file=sys.stdout)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            tqdm.write("epoch %02d batch id %04d/%d loss %f train acc %f" %(e+1, batch_id+1, n_batch, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    train_acc /= batch_id + 1
    print("epoch %02d train acc %f" %(e+1, train_acc))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(valid_dataloader, file=sys.stdout)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        valid_acc += calc_accuracy(out, label)
    valid_acc /=  batch_id+1
    print("epoch %02d valid acc %f" %(e+1, valid_acc))
    torch.save(model, 'ver_3_%02d_%.04f_%.04f.model' %(e+1, train_acc, valid_acc))

epoch 01 batch id 0001/3105 loss 3.090259 train acc 0.000000
epoch 01 batch id 0201/3105 loss 2.972554 train acc 0.053897
epoch 01 batch id 0401/3105 loss 2.639045 train acc 0.068579
epoch 01 batch id 0601/3105 loss 2.756540 train acc 0.105380
epoch 01 batch id 0801/3105 loss 2.507110 train acc 0.153142
epoch 01 batch id 1001/3105 loss 1.937168 train acc 0.240926
epoch 01 batch id 1201/3105 loss 0.536893 train acc 0.307799
epoch 01 batch id 1401/3105 loss 0.522561 train acc 0.375922
epoch 01 batch id 1601/3105 loss 1.553600 train acc 0.427753
epoch 01 batch id 1801/3105 loss 2.545682 train acc 0.467426
epoch 01 batch id 2001/3105 loss 1.301041 train acc 0.487923
epoch 01 batch id 2201/3105 loss 0.614586 train acc 0.510147
epoch 01 batch id 2401/3105 loss 0.292521 train acc 0.535402
epoch 01 batch id 2601/3105 loss 0.180351 train acc 0.558183
epoch 01 batch id 2801/3105 loss 0.013120 train acc 0.578365
epoch 01 batch id 3001/3105 loss 0.300234 train acc 0.588359
100%|██████████| 3105/31

epoch 08 batch id 0801/3105 loss 0.326475 train acc 0.826259
epoch 08 batch id 1001/3105 loss 1.469251 train acc 0.837163
epoch 08 batch id 1201/3105 loss 0.003693 train acc 0.842770
epoch 08 batch id 1401/3105 loss 0.005497 train acc 0.852605
epoch 08 batch id 1601/3105 loss 0.292724 train acc 0.844576
epoch 08 batch id 1801/3105 loss 1.847094 train acc 0.842217
epoch 08 batch id 2001/3105 loss 0.003793 train acc 0.837331
epoch 08 batch id 2201/3105 loss 0.004629 train acc 0.827275
epoch 08 batch id 2401/3105 loss 0.004449 train acc 0.823893
epoch 08 batch id 2601/3105 loss 0.004800 train acc 0.824298
epoch 08 batch id 2801/3105 loss 0.002405 train acc 0.829466
epoch 08 batch id 3001/3105 loss 0.016127 train acc 0.828502
100%|██████████| 3105/3105 [12:26<00:00,  4.16it/s]
epoch 08 train acc 0.831079
100%|██████████| 409/409 [00:31<00:00, 13.15it/s]
epoch 08 valid acc 0.804808
epoch 09 batch id 0001/3105 loss 0.017016 train acc 1.000000
epoch 09 batch id 0201/3105 loss 0.009073 train a

epoch 15 batch id 1601/3105 loss 0.008915 train acc 0.930460
epoch 15 batch id 1801/3105 loss 2.327178 train acc 0.931520
epoch 15 batch id 2001/3105 loss 0.002596 train acc 0.927620
epoch 15 batch id 2201/3105 loss 0.002473 train acc 0.927079
epoch 15 batch id 2401/3105 loss 0.008666 train acc 0.928710
epoch 15 batch id 2601/3105 loss 0.002198 train acc 0.930027
epoch 15 batch id 2801/3105 loss 0.002592 train acc 0.931989
epoch 15 batch id 3001/3105 loss 0.002672 train acc 0.931134
100%|██████████| 3105/3105 [12:23<00:00,  4.18it/s]
epoch 15 train acc 0.932206
100%|██████████| 409/409 [00:31<00:00, 13.17it/s]
epoch 15 valid acc 0.818663
epoch 16 batch id 0001/3105 loss 0.002671 train acc 1.000000
epoch 16 batch id 0201/3105 loss 0.001205 train acc 0.912106
epoch 16 batch id 0401/3105 loss 0.002566 train acc 0.923525
epoch 16 batch id 0601/3105 loss 0.005648 train acc 0.924847
epoch 16 batch id 0801/3105 loss 0.001227 train acc 0.930087
epoch 16 batch id 1001/3105 loss 0.001203 train a

epoch 22 batch id 2401/3105 loss 0.002976 train acc 0.954047
epoch 22 batch id 2601/3105 loss 0.002528 train acc 0.954505
epoch 22 batch id 2801/3105 loss 0.003108 train acc 0.955730
epoch 22 batch id 3001/3105 loss 0.002361 train acc 0.954682
100%|██████████| 3105/3105 [12:21<00:00,  4.19it/s]
epoch 22 train acc 0.955072
100%|██████████| 409/409 [00:31<00:00, 13.18it/s]
epoch 22 valid acc 0.829666
epoch 23 batch id 0001/3105 loss 0.003315 train acc 1.000000
epoch 23 batch id 0201/3105 loss 0.002285 train acc 0.940299
epoch 23 batch id 0401/3105 loss 0.001366 train acc 0.947215
epoch 23 batch id 0601/3105 loss 0.004089 train acc 0.948697
epoch 23 batch id 0801/3105 loss 0.001115 train acc 0.952143
epoch 23 batch id 1001/3105 loss 0.000732 train acc 0.953713
epoch 23 batch id 1201/3105 loss 0.001396 train acc 0.955037
epoch 23 batch id 1401/3105 loss 0.002381 train acc 0.955865
epoch 23 batch id 1601/3105 loss 0.007003 train acc 0.956277
epoch 23 batch id 1801/3105 loss 2.062234 train a

# Predict

In [15]:
from sklearn.metrics import recall_score, precision_score, f1_score

In [16]:
model = torch.load('ver_2_16_0.9530_0.8333.model')
label_w2i = pickle.load(open('ver_2.w2i', 'rb'))
label_i2w = {label_w2i[l]: l for l in label_w2i}

In [17]:
def get_pred(X):
    _, indices = torch.max(X, 1)
    return indices.cpu().numpy().tolist()

In [26]:
valid_acc = 0
test_pred = []
model.eval()
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader, file=sys.stdout)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    test_pred.append(get_pred(out))
test_pred = np.concatenate(test_pred)

100%|██████████| 380/380 [00:28<00:00, 13.38it/s]


In [27]:
test_true = np.array([d[2] for d in test_dataset])

In [28]:
test_acc = (test_pred == test_true).sum() / len(test_true)
test_precision = [precision_score(test_true, test_pred, labels=[i], average='macro') for i in label_i2w]
test_recall = [recall_score(test_true, test_pred, labels=[i], average='macro') for i in label_i2w]
test_f1 = [f1_score(test_true, test_pred, labels=[i], average='macro') for i in label_i2w]

In [29]:
print('| Label | Precision | Recall | F1 |')
print('|-------|-----------|--------|----|')
for i, (precision, recall, f1) in enumerate(zip(test_precision, test_recall, test_f1)):
    print('| %s | %.4f | %.4f | %.4f |' %(label_i2w[i], precision, recall, f1))
print('| %s | %.4f | %.4f | %.4f |' %(
    'Total', 
    precision_score(test_true, test_pred, average='macro'), 
    recall_score(test_true, test_pred, average='macro'), 
    f1_score(test_true, test_pred, average='macro'), 
))

print('Test Accuracy : %.4f' %test_acc)

| Label | Precision | Recall | F1 |
|-------|-----------|--------|----|
| 가족 | 0.7439 | 0.5755 | 0.6489 |
| 건강/다이어트 | 0.7698 | 0.8818 | 0.8220 |
| 계절/날씨 | 0.9200 | 0.8647 | 0.8915 |
| 꿈(목표) | 0.6134 | 0.7766 | 0.6854 |
| 먹거리 | 0.9209 | 0.8996 | 0.9102 |
| 반려동물 | 0.9590 | 0.9832 | 0.9710 |
| 방송/연예 | 0.9101 | 0.6207 | 0.7380 |
| 선물 | 0.9224 | 0.9068 | 0.9145 |
| 성격 | 0.9256 | 0.8960 | 0.9106 |
| 스포츠/레저 | 0.8471 | 0.8110 | 0.8287 |
| 아르바이트 | 0.7403 | 0.9828 | 0.8444 |
| 여행지(국내/해외) | 0.8508 | 0.9277 | 0.8876 |
| 연애/결혼 | 0.6902 | 0.9137 | 0.7864 |
| 영화 | 0.8424 | 0.9553 | 0.8953 |
| 회사/학교 | 0.8205 | 0.6809 | 0.7442 |
| Total | 0.8318 | 0.8451 | 0.8319 |
Test Accuracy : 0.8362


In [35]:
test_pred

array([13, 13, 13, ...,  6,  6,  6])

In [15]:
raw_data = [
    ('혹시 마블 좋아하세요?', '네. 최근에 스파이더맨 봤어요.'),
    ('점심 뭐 드셨어요?', '근처에서 마라탕 먹었어요.')
]
data = BERTDataset([d + (0,) for d in raw_data], 0, 1, 2, tok, max_len, True, False)
dataloader = DataLoader(data, batch_size=batch_size, num_workers=5)

100%|██████████| 2/2 [00:00<00:00, 91.12it/s]


In [16]:
model.eval()
outputs = []
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    outputs.append(out.cpu().detach().numpy())

result = np.concatenate(outputs)

/home/svclaw2000/Dialog-Classification-Using-KoBERT/venv_kcc/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
result

array([[-1.6374834 , -1.3215848 ,  0.12254745, -2.5353487 , -0.89550805,
        -1.4683422 ,  1.9166542 , -2.0298762 , -1.7553192 ,  1.1083449 ,
        -1.3344933 , -0.5012208 ,  0.7741338 ,  8.988237  , -0.67473793],
       [ 0.14410633,  0.48958924, -0.43005556, -1.36853   , 10.405009  ,
        -1.5924621 , -1.6080155 ,  0.31998512, -2.137295  , -0.46647248,
        -0.10372277,  2.2267296 , -1.6280773 , -1.0927978 , -1.1962178 ]],
      dtype=float32)

In [62]:
full_length = list(map(lambda x: len(x[2] + x[3]), raw_train_dataset))

In [65]:
np.argmax(full_length)

18082